In [2]:
suppressMessages(library(tidyverse)) #loading libraries, suppresses warning messages.
suppressMessages(library(tidymodels)) 
suppressMessages(library(repr))

set.seed(4747) #setting seed for reproducibility's sake. 47 is hexadecimal for g, and our group number "g47"

## Project Proposal

In [2]:
url <- "https://drive.google.com/uc?export=download&id=1_MECmUXZuuILYeEOfonSGqodW6qVdhsS"
dir.create("data")
download.file(url, "data/top500players.csv") #creates directory "data" and downloads the data used into the folder

### RUN THIS BLOCK ONLY ONCE 

In [3]:
player_data <- read_csv("data/top500players.csv", show_col_types = FALSE)
head(player_data)

New names:
* `` -> ...1



...1,Age,Country,Plays,Wikipedia,Current Rank,Best Rank,Name,Backhand,Prize Money,⋯,Facebook,Twitter,Nicknames,Grand Slams,Davis Cups,Web Site,Team Cups,Olympics,Weeks at No. 1,Tour Finals
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
0,26 (25-04-1993),Brazil,Right-handed,Wikipedia,378 (97),363 (04-11-2019),Oscar Jose Gutierrez,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1,18 (22-12-2001),United Kingdom,Left-handed,Wikipedia,326 (119),316 (14-10-2019),Jack Draper,Two-handed,"$59,040",⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,32 (03-11-1987),Slovakia,Right-handed,Wikipedia,178 (280),44 (14-01-2013),Lukas Lacko,Two-handed,"US$3,261,567",⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,21 (29-05-1998),"Korea, Republic of",Right-handed,Wikipedia,236 (199),130 (10-04-2017),Duck Hee Lee,Two-handed,"$374,093",⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,27 (21-10-1992),Australia,Right-handed,Wikipedia,183 (273),17 (11-01-2016),Bernard Tomic,Two-handed,"US$6,091,971",⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,22 (11-02-1997),Poland,Right-handed,Wikipedia,31 (1398),31 (20-01-2020),Hubert Hurkacz,Two-handed,"$1,517,157",⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [4]:
colnames(player_data) <- make.names(colnames(player_data))

player_data_tidy <- player_data |>
                        separate(Age, c("Age", NA), sep = " ") |> #Removes the DOB next to the Age
                        separate(Current.Elo.Rank, c("Current.Rank", NA), sep = " ") |> #Removes the elo next to the ranks
                        select(Age, Name, Seasons, Current.Rank) |>
                        na.omit() |>
                        mutate(Age = as.numeric(Age)) |>
                        mutate(Current.Rank = as.numeric(Current.Rank))

head(player_data_tidy)

Age,Name,Seasons,Current.Rank
<dbl>,<chr>,<dbl>,<dbl>
32,Lukas Lacko,14,144
27,Bernard Tomic,11,100
22,Hubert Hurkacz,5,33
25,Sanjar Fayziev,5,189
19,Felix Auger Aliassime,3,51
23,Daniil Medvedev,5,5
